In [11]:
import pkg_resources
from symspellpy import SymSpell, Verbosity
import time
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np 
import pandas as pd

In [12]:
sym_spell = SymSpell(max_dictionary_edit_distance=3, prefix_length=7)
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

True

In [13]:
with open("DO_NOT_CHECK.txt","r") as f:
    DO_NOT_CHECK = f.read()
    DO_NOT_CHECK.split(" ")

In [14]:
def fix_sentence(sentence,DO_NOT_CHECK=DO_NOT_CHECK):
    fixed = []
    for word in sentence.split(" "):
        word = word.lower()
        if word in DO_NOT_CHECK:
            fixed.append(word)
        else:
            suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=3, include_unknown=True)
            fixed.append(suggestions[0].__str__().split(",")[0])
    return fixed

In [15]:
text = " ".join(fix_sentence('Whre can I fnid informion about my past pruchases'))
text

'were can i find information about my past purchases'

In [27]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2').cpu()
model.eval()
1+1

2

In [28]:
data = pd.read_csv("input and responses.csv", low_memory=False)
data

,INPUT_TEXT,RESPONSE
0,Can I speak to the Human,"Yes, please wait while we are connecting you t..."
1,Hi,Hello. How can I help you?
2,Hello,Hello. How can I help you?
3,Good Morning,Hello. How can I help you?
4,Good Afternoon,Hello. How can I help you?
5,Greetings,Hello. How can I help you?
6,I want to cancel my subscription,To cancel you subscription you need to go to <...
7,I want to renew a subscription,To renew you subscription you need to go to <l...
8,How can I edit my personal details,"In order to edit your personal details, please..."
9,Where can I find information on my payments,All information on past and future payemnts ca...


In [29]:
inp_res_dicto = {}
for inp,res in zip(data.INPUT_TEXT, data.RESPONSE):
    inp_res_dicto.update({inp:res})
np.save('inp_res_dicto.npy', inp_res_dicto) 

In [30]:
temp = model.encode(data.INPUT_TEXT.str.lower())
temp

array([[-4.01056744e-03,  3.86110209e-02, -4.65880008e-03, ...,
         6.28489703e-02, -5.12792207e-02, -5.95645159e-02],
       [-9.05224085e-02,  4.04279791e-02,  2.38645244e-02, ...,
         6.81075826e-02,  1.78261809e-02,  1.12706386e-02],
       [-6.28017783e-02,  5.49701154e-02,  5.21321073e-02, ...,
         5.08378744e-02,  5.15148491e-02,  7.04126433e-03],
       ...,
       [ 4.58948416e-05,  2.48811394e-02, -1.71080901e-04, ...,
        -3.21780033e-02, -2.27981992e-02,  1.07641891e-02],
       [-2.86385063e-02, -4.95260954e-03, -1.91149879e-02, ...,
        -1.12944111e-01, -2.32961942e-02, -1.23223728e-02],
       [ 3.68098234e-04, -1.04472116e-01, -6.10006899e-02, ...,
         4.62266244e-02,  3.98446480e-03,  1.12122661e-02]], dtype=float32)

In [31]:
inp_emb_dicto = {}
for i,inp in enumerate(data.INPUT_TEXT.tolist()):
    inp_emb_dicto.update({inp:temp[i,:]})
np.save('inp_emb_dicto.npy', inp_emb_dicto) 

In [33]:
embeddings1 = model.encode(" ".join(fix_sentence('Whre can I fnid informion about my past pruchases')))
best_score = 0
best_key = None
for key in dicto.keys():
    temp_score = util.pytorch_cos_sim(embeddings1, dicto[key])
    if temp_score > best_score:
        best_key = key
        best_score = temp_score
best_score,best_key


(tensor([[0.9827]]), 'Where can I find information about my past purchases')